In [12]:
import os
import glob
import uuid
import datetime
import warnings

from itertools import product
from multiprocessing import Pool

import tqdm
import netCDF4
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

from numba import jit, int32
warnings.simplefilter('ignore')

# File list

In [13]:
@jit(nopython=True)
def convective_radius(ze_bkg, area_relation):
    """
    Given a mean background reflectivity value, we determine via a step
    function what the corresponding convective radius would be
    Higher background reflectivitives are expected to have larger convective
    influence on surrounding areas, so a larger convective radius would be
    prescribed
    """
    if (area_relation == 0):
        if (ze_bkg < 30):
            conv_rad = 1000.
        elif (ze_bkg >= 30) & (ze_bkg < 35.):
            conv_rad = 2000.
        elif (ze_bkg >= 35.) & (ze_bkg < 40.):
            conv_rad = 3000.
        elif (ze_bkg >= 40.) & (ze_bkg < 45.):
            conv_rad = 4000.
        else:
            conv_rad = 5000.

    if (area_relation == 1):
        if (ze_bkg < 25):
            conv_rad = 1000.
        elif (ze_bkg >= 25) & (ze_bkg < 30.):
            conv_rad = 2000.
        elif (ze_bkg >= 30.) & (ze_bkg < 35.):
            conv_rad = 3000.
        elif (ze_bkg >= 35.) & (ze_bkg < 40.):
            conv_rad = 4000.
        else:
            conv_rad = 5000.

    if (area_relation == 2):
        if (ze_bkg < 20):
            conv_rad = 1000.
        elif (ze_bkg >= 20) & (ze_bkg < 25.):
            conv_rad = 2000.
        elif (ze_bkg >= 25.) & (ze_bkg < 30.):
            conv_rad = 3000.
        elif (ze_bkg >= 30.) & (ze_bkg < 35.):
            conv_rad = 4000.
        else:
            conv_rad = 5000.

    if (area_relation == 3):
        if (ze_bkg < 40):
            conv_rad = 0.
        elif (ze_bkg >= 40) and (ze_bkg < 45.):
            conv_rad = 1000.
        elif (ze_bkg >= 45.) and (ze_bkg < 50.):
            conv_rad = 2000.
        elif (ze_bkg >= 50.) and (ze_bkg < 55.):
            conv_rad = 6000.
        else:
            conv_rad = 8000.

    return conv_rad

@jit(nopython=True)
def peakedness(ze_bkg, peak_relation):
    """
    Given a background reflectivity value, we determine what the necessary
    peakedness (or difference) has to be between a grid point's reflectivity
    and the background reflectivity in order for that grid point to be labeled
    convective
    """
    if (peak_relation == 0):
        if (ze_bkg < 0.):
            peak = 10.
        elif (ze_bkg >= 0.) and (ze_bkg < 42.43):
            peak = 10. - ze_bkg ** 2 / 180.
        else:
            peak = 0.

    elif (peak_relation == 1):
        if (ze_bkg < 0.):
            peak = 14.
        elif (ze_bkg >= 0.) and (ze_bkg < 42.43):
            peak = 14. - ze_bkg ** 2 / 180.
        else:
            peak = 4.

    return peak

@jit(nopython=True)
def steiner_classification(refl, x, y, dx, dy, intense=42, peak_relation=0,
                           area_relation=1, bkg_rad=11000, use_intense=True):
    """
    We perform the Steiner et al. (1995) algorithm for echo classification
    using only the reflectivity field in order to classify each grid point
    as either convective, stratiform or undefined. Grid points are classified
    as follows,
    0 = Undefined
    1 = Stratiform
    2 = Convective

    Parameters:
    ===========
    refl: ndarray
        Reflectivity slice (2D Cartesion grid).
    x: ndarray
        x-coordinates
    y: ndarray
        y-coordinates
    dx: float
        x-coordinates resolution
    dy: float
        y-coordinates resolution
    intense: float
        Value above which a pixel is consider convective no matter what.

    Returns:
    ========
    sclass: ndarray <int>
        Convective/Stratiform classification, same size as refl.
    """

    sclass = np.zeros(refl.shape, dtype=int32)
    ny, nx = refl.shape

    for i in range(0, nx):
        # Get stencil of x grid points within the background radius
        imin = np.max(np.array([1, (i - bkg_rad / dx)], dtype=int32))
        imax = np.min(np.array([nx, (i + bkg_rad / dx)], dtype=int32))

        for j in range(0, ny):
            # First make sure that the current grid point has not already been classified.
            # This can happen when grid points within the convective radius of a previous
            # grid point have also been classified
            if ~np.isnan(refl[j, i]) & (sclass[j, i] == 0):
                # Get stencil of y grid points within the background radius
                jmin = np.max(np.array([1, (j - bkg_rad / dy)], dtype=int32))
                jmax = np.min(np.array([ny, (j + bkg_rad / dy)], dtype=int32))

                n = 0
                sum_ze = 0

                # Calculate the mean background reflectivity for the current grid point,
                # which will be used to determine the convective radius and the required peakedness

                for l in range(imin, imax):
                    for m in range(jmin, jmax):
                        rad = np.sqrt((x[l] - x[i]) ** 2 + (y[m] - y[j]) ** 2)

                        # The mean background reflectivity will first be computed in
                        # linear units, i.e. mm^6/m^3, then converted to decibel units.
                        if rad <= bkg_rad:
                            n += 1
                            sum_ze += 10. ** (refl[m, l] / 10.)

                ze_bkg = 10.0 * np.log10(sum_ze / n)

                # Now get the corresponding convective radius knowing the mean background reflectivity
                conv_rad = convective_radius(ze_bkg, area_relation)

                # Now we want to investigate the points surrounding the current
                # grid point that are within the convective radius, and whether
                # they too are convective, stratiform or undefined

                # Get stencil of x and y grid points within the convective radius
                lmin = np.max(np.array([1, int(i - conv_rad / dx)]))
                lmax = np.min(np.array([nx, int(i + conv_rad / dx)]))
                mmin = np.max(np.array([1, int(j - conv_rad / dy)]))
                mmax = np.min(np.array([ny, int(j + conv_rad / dy)]))

                if use_intense and (refl[j, i] >= intense):
                    sclass[j, i] = 2

                    for l in range(lmin, lmax):
                        for m in range(mmin, mmax):
                            if not np.isnan(refl[j, i]):
                                rad = np.sqrt((x[l] - x[i]) ** 2 + (y[m] - y[j]) ** 2)

                                if rad <= conv_rad:
                                    sclass[m, l] = 2

                else:
                    peak = peakedness(ze_bkg, peak_relation)

                    if refl[j, i] - ze_bkg >= peak:
                        sclass[j, i] = 2

                        for l in range(imin, imax):
                            for m in range(jmin, jmax):
                                if not np.isnan(refl[j, i]):
                                    rad = np.sqrt((x[l] - x[i]) ** 2 + (y[m] - y[j]) ** 2)

                                    if rad <= conv_rad:
                                        sclass[m, l] = 2

                    else:
                        # If by now the current grid point has not been classified as convective
                        # by either the intensity criteria or the peakedness criteria, then it must be stratiform
                        sclass[j, i] = 1

    return sclass

In [14]:
def get_file_list(input_dir):
    '''
    Generate file list.
    
    Parameter:
    ===========
    input_dir: str
        Path to input data directory
        
    Returns:
    ========
    flist: list
        Input file list.
    '''
    flist = sorted(glob.glob(os.path.join(input_dir, '*.nc')))
    if len(flist) == 144:
        return flist
    
    flist = [None] * 144
    for cnt, (h, m) in enumerate(product(range(0, 24), range(0, 6))):
            infiles = glob.glob(os.path.join(input_dir, f'*_{h:02}{m}*.nc'))
            if len(infiles) == 0:
                continue

            infile = infiles[0]
            if os.path.isfile(infile):
                flist[cnt] = infile
    
    return flist

In [15]:
def update_metadata(gnrl_meta):
    '''
    Correct general metadata dictionnary.
    
    Parameters:
    ===========
    gnrl_meta: dict
        General metadata dictionnary.
        
    Returns:
    ========
    gnrl_meta: dict
        Updated general metadata dictionnary.
    '''
    maxlon = '132.3856852067545'
    minlon = '129.70320368213441'
    maxlat = '-10.941777804922253'
    minlat = '-13.552905831511362'
    latres = '0.0225'

    origin_altitude = '50'
    origin_latitude = '-12.249'
    origin_longitude = '131.044'
    projection = 'Azimuthal equidistant projection'

    obsolete_keys = ['original_format', 'n_gates_vary', 'driver', 'start_datetime', 
                     'start_time', 'end_datetime', 'end_time', 'scan_name', 'scan_id', 
                     'ray_times_increase', 'Conventions', 'Sub_conventions']
    for key in obsolete_keys:
        try:
            gnrl_meta.pop(key)
        except KeyError:
            pass

    gnrl_meta['version'] = '2018.06_level2'
    gnrl_meta['created'] = datetime.datetime.now().isoformat()
    gnrl_meta['uuid'] = str(uuid.uuid4())
    gnrl_meta['processing_level'] = 'L2'

    gnrl_meta['geospatial_bounds'] = f"({minlon}, {maxlon}, {minlat}, {maxlat})"
    gnrl_meta['geospatial_lat_min'] = minlat
    gnrl_meta['geospatial_lat_max'] = maxlat
    gnrl_meta['geospatial_lat_units'] = "degrees_north"
    gnrl_meta['geospatial_lat_resolution'] = latres
    gnrl_meta['geospatial_lon_min'] = minlon
    gnrl_meta['geospatial_lon_max'] = maxlon
    gnrl_meta['geospatial_lon_units'] = "degrees_east"
    gnrl_meta['geospatial_lon_resolution'] = latres
#     gnrl_meta['geospatial_vertical_min'] = '0'
#     gnrl_meta['geospatial_vertical_max'] = '20000'
#     gnrl_meta['geospatial_vertical_resolution'] = '500'
#     gnrl_meta['geospatial_vertical_units'] = "meters"
    gnrl_meta['origin_latitude'] = origin_latitude
    gnrl_meta['origin_longitude'] = origin_longitude
    gnrl_meta['origin_altitude'] = origin_altitude
    gnrl_meta['geospatial_projection'] = projection
    
    return gnrl_meta

In [16]:
def read_data(input_file, data_key='radar_estimated_rain_rate', level=0, bad=-9999):
    '''
    Read netCDF4 file, data, data metadata, and file metadata.
    
    Parameters:
    ===========
    input_file: str
        Input file name.
    data_key: str
        Data moment name
        
    Returns:
    ========
    data: ndarray
        Data
    data_meta: dict
        Data metadata
    gnrl_meta: dict
        File metadata
    '''
    try:
        with netCDF4.Dataset(input_file) as ncid:
            data = np.squeeze(ncid[data_key][:, level, :, :]).filled(bad)
            data_meta_nc = ncid[data_key]

            data_meta = dict()
            for key in data_meta_nc.ncattrs():
                data_meta[key] = data_meta_nc.getncattr(key)

            gnrl_meta = dict()
            for key in ncid.ncattrs():
                gnrl_meta[key] = ncid.getncattr(key)
    except Exception:
        print(input_file)
        raise
            
    return data, data_meta, gnrl_meta

In [17]:
def mkdir(mydir):
    if os.path.exists(mydir):
        return None
    
    try:
        os.mkdir(mydir)
    except FileExistsError:
        return None
    
    return None        

In [18]:
def make_dailys(inargs):
    x, y, date, INDIR, OUTDIR, MOMENT_NAME, RMAX, level, bad, XDIM, YDIM = inargs
    
    if MOMENT_NAME == 'steiner_echo_classification':
        is_steiner = True
    else:
        is_steiner = False
    
    # Check if input dir exits.
    indir = os.path.join(INDIR, str(date.year), date.strftime('%Y%m%d'))
    if not os.path.exists(indir):
        print(f'Input dir {indir} does not exist.')
        return None
    flist = get_file_list(indir)
    
    # Generate output file name
    outdir = os.path.join(OUTDIR, MOMENT_NAME.upper())
    mkdir(outdir)
    outfilename = os.path.join(outdir, 'CPOL_{}_{}.nc'.format(MOMENT_NAME.upper(), date.strftime('%Y%m%d')))
    if os.path.exists(outfilename):
        print(f'Output file {outfilename} already exists. Doing nothing.')
        return None
    
    if XDIM is None:
        XDIM = len(x)
    if YDIM is None:
        YDIM = len(y)
    
    X, Y = np.meshgrid(x, y)
    R = np.sqrt(X ** 2 + Y ** 2)
    
    # Read data
    RAIN_TOT = np.zeros((144, XDIM, YDIM))
    for cnt, infile in enumerate(flist):
        if infile is None:
            RAIN_TOT[cnt, :, :] = np.NaN
            continue
        if is_steiner:
            try:
                rain, rain_meta, gnrl_meta = read_data(infile, 'reflectivity', level=level, bad=bad)
            except Exception:
                print(infile)
                raise
        else:
            rain, rain_meta, gnrl_meta = read_data(infile, MOMENT_NAME, level=level, bad=bad)
        rain[R >= RMAX] = np.NaN
        RAIN_TOT[cnt, :, :] = rain

    RAIN_TOT = np.ma.masked_where(np.isnan(RAIN_TOT), RAIN_TOT)
    gnrl_meta = update_metadata(gnrl_meta)   
    
    # Generate time dimension
    st = np.array(date, dtype=np.datetime64)
    ed = np.array(date, dtype=np.datetime64) +  np.timedelta64(1,'D')
    dtime = np.arange(st, ed, np.timedelta64(10, 'm'))  # Every 10 minutes
    time_unit = f'seconds since {str(dtime[0])}'
    time = netCDF4.date2num(dtime.tolist(), time_unit).astype(np.int32)
    
    # Write data
    with netCDF4.Dataset(outfilename, 'w') as ncid:
        ncid.createDimension('time', 144)
        ncid.createDimension("longitude", XDIM)
        ncid.createDimension("latitude", YDIM)

        mymoment = ncid.createVariable(MOMENT_NAME, RAIN_TOT.dtype, ("time", "latitude", "longitude"), zlib=True)

        nctime = ncid.createVariable('time', time.dtype, 'time')
        nclon = ncid.createVariable('longitude', LON.dtype, ('longitude'))
        nclat = ncid.createVariable('latitude', LAT.dtype, ('latitude'))
        nclon[:] = LON
        nclon.units = 'degrees_east'
        nclat[:] = LAT
        nclat.units = 'degrees_north'
#         ncx = ncid.createVariable('x', x.dtype, 'x')
#         ncy = ncid.createVariable('y', y.dtype, 'y')

        nctime[:] = time
        nctime.units = time_unit
#         ncx[:] = x
#         ncx.units = 'km'
#         ncy[:] = y
#         ncy.units = 'km'
        
        # Write attributes
        mymoment[:] = RAIN_TOT
        for k, v in rain_meta.items():
            if k == '_FillValue':
                continue
            try:
                mymoment.setncattr(str(k), str(v))
            except AttributeError:
                print(k)
                print(v)
                print(type(k))
                print(type(v))
                raise

        for k, v  in gnrl_meta.items():
            ncid.setncattr(k, str(v))
            
    return None

In [19]:
# XDIM = 117
# YDIM = 117
# RMAX = 140
# MOMENT_NAME = 'steiner_echo_classification'
# OUTDIR = '/g/data2/rr5/vhl548/NEW_CPOL_level_2'
# INDIR = '/g/data2/rr5/vhl548/CPOL_level_1b/GRIDDED/GRID_150km_2500m/'

# x = np.linspace(-145, 145, XDIM, dtype=np.float32)
# y = np.linspace(-145, 145, YDIM, dtype=np.float32)

XDIM = 141
YDIM = 141
RMAX = 140
MOMENT_NAME = 'steiner_echo_classification'
OUTDIR = '/g/data2/rr5/vhl548/NEW_CPOL_level_2_1km'
INDIR = '/g/data2/rr5/vhl548/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

x = np.linspace(-70, 70, XDIM, dtype=np.float32)
y = np.linspace(-70, 70, YDIM, dtype=np.float32)

In [20]:
if XDIM == 117:
    fdlatlon = '/g/data2/rr5/vhl548/CPOL_level_1b/GRIDDED/GRID_150km_2500m/2017/20170304/CPOL_20170304_0000_GRIDS_2500m.nc'
else:
    fdlatlon = '/g/data2/rr5/vhl548/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2017/20170304/CPOL_20170304_0000_GRIDS_1000m.nc'

with netCDF4.Dataset(fdlatlon) as ncid:
    mylat = np.squeeze(ncid['latitude'][:].filled(np.NaN))
    mylon = np.squeeze(ncid['longitude'][:].filled(np.NaN))

if XDIM == 117:
    LAT = mylat[0, :, 58]
    LON = mylon[0, 58, :]
else:
    LAT = mylat[0, :, 70]
    LON = mylon[0, 70, :]

In [21]:
date_range = pd.date_range('19981206', '20170502')
# args_list = [None] * len(date_range)
args_list = []
for cnt, dt in enumerate(date_range):
    indir = os.path.join(INDIR, str(dt.year), dt.strftime('%Y%m%d'))
    if not os.path.exists(indir):
        continue
    args_list.append((x, y, dt, INDIR, OUTDIR, MOMENT_NAME, RMAX, 0, 0, XDIM, YDIM))

In [22]:
import time
time.sleep(1500)
with Pool(16) as pool:
    rslt = list(tqdm.tqdm_notebook(pool.imap(make_dailys, args_list), total=len(args_list)))